# <center> ⚔️6.1 - Блендинг </center>

<div class="alert alert-info">

Основная идея данной техники заключается в том, что взять от каждого алгоритма лучшее и совместить несколько разных ML моделей в одну. За счет такого объединения увеличивается обобщающая способность финальной модели и качество улучшается. Помимо этого ваша модель становится сильно более стабильной, что позволяет на слететь на приватном лидерборде. Особенно хорошо накидывает блендинг, если модели, которые совмещаете имеют разную природу: допустим, нейронные сети, kNN и решающие деревья, так как в этом случае они выучивают разные зависимости и хорошо дополняют друг друга.

# <center> <img src = '../images/blending.jpg' width=750>

## Импортируем библиотеки

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import lightgbm as lgbm
import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor

## Считываем данные

In [3]:
data = pd.read_csv('../data/car_train.csv')
data.head()

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,108.53,another_bug
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,35.20,electro_bug
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,38.62,gear_stick
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,30.34,engine_fuel
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,30.45,engine_fuel


In [4]:
data['car_id'].nunique()

2337

In [5]:
data.shape

(2337, 10)

### Заменим категориальные признаки на числовые

In [6]:
cat_cols = ['model', 'car_type', 'fuel_type']
for col in cat_cols:
    data[col] = data[col].replace(np.unique(data[col]),
                             np.arange(data[col].nunique()))
    data[col] = data[col].astype('category')
data.head()

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
0,y13744087j,8,1,1,3.78,2015,76163,2021,108.53,another_bug
1,O41613818T,23,1,1,3.90,2015,78218,2021,35.20,electro_bug
2,d-2109686j,16,3,1,6.30,2012,23340,2017,38.62,gear_stick
3,u29695600e,12,0,1,4.04,2011,1263,2020,30.34,engine_fuel
4,N-8915870N,16,3,1,4.70,2012,26428,2017,30.45,engine_fuel


### Разделим выборку на валидационную и обучающую

In [7]:
COLS_BEST = data.drop(columns=['car_id', 'target_reg', 'target_class']).columns

In [8]:
X_train, X_val, y_train, y_val = train_test_split(data.drop(['target_class'], axis=1)[COLS_BEST], 
                                                    data['target_reg'],
                                                    test_size=.25,
                                                    random_state=42)
X_train.shape, X_val.shape

((1752, 7), (585, 7))

## Обучаем CatBoost

In [347]:
text_cols = []

In [348]:
params_cat = {'n_estimators' : 700,
          'learning_rate': .03,
          'depth' : 3,
          'use_best_model': True,
          'cat_features' : cat_cols,
          'text_features': text_cols,
          # 'train_dir' : '/home/jovyan/work/catboost',
          # 'border_count' : 64,
          # 'l2_leaf_reg' : 1,
          # 'bagging_temperature' : 2,
          # 'rsm' : .1,
          # 'loss_function': 'MultiClass',
          # 'auto_class_weights' : 'Balanced', #try not balanced
          'random_state': 42,
          # 'custom_metric' : ['AUC', 'MAP']
         }

cat_model = CatBoostRegressor(**params_cat)

In [349]:
cat_model.fit(X_train, y_train, verbose=100, eval_set=(X_val, y_val), early_stopping_rounds=100)

0:	learn: 17.6161100	test: 17.6550615	best: 17.6550615 (0)	total: 3.22ms	remaining: 2.25s
100:	learn: 17.1615819	test: 17.1692060	best: 17.1682442 (99)	total: 79.2ms	remaining: 470ms
200:	learn: 17.0286130	test: 17.1696516	best: 17.1593349 (113)	total: 138ms	remaining: 344ms
Stopped by overfitting detector  (100 iterations wait)

bestTest = 17.1593349
bestIteration = 113

Shrink model to first 114 iterations.


In [350]:
mean_squared_error(cat_model.predict(X_val), y_val)

294.44277076934935

In [351]:
mean_squared_error(np.ones(len(y_val)) * y_val.mean(), y_val)

312.2798719631821

In [352]:
submit = pd.DataFrame({'target': cat_model.predict(X_val).reshape(-1)})
submit.to_csv('../data/blending/regression/catboost_preds.csv', index=False)
submit.head()

,target
0,65.927922
1,44.248116
2,43.807966
3,45.661121
4,43.653036


In [353]:
submit.shape

(585, 1)

## Обучаем LightGbm

In [354]:
import warnings
warnings.filterwarnings("ignore")

In [355]:
params_lgbm = {'num_leaves': 887,
               'n_estimators': 480,
               'max_depth': 7,
               'min_child_samples': 1073,
               'learning_rate': 0.05348257149091985,
               'min_data_in_leaf': 2,
               'feature_fraction': 0.9529134909800754,
               'categorical_feature': cat_cols
              }

In [356]:
lgbm_model = lgbm.LGBMRegressor(**params_lgbm)

In [357]:
lgbm_model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=100, verbose=25)

[LightGBM] [Warning] min_data_in_leaf is set=2, min_child_samples=1073 will be ignored. Current value: min_data_in_leaf=2
[LightGBM] [Warning] feature_fraction is set=0.9529134909800754, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9529134909800754
[25]	valid_0's l2: 308.382
[50]	valid_0's l2: 315.925
[75]	valid_0's l2: 325.238
[100]	valid_0's l2: 329.397


LGBMRegressor(categorical_feature=['model', 'car_type', 'fuel_type'],
              feature_fraction=0.9529134909800754,
              learning_rate=0.05348257149091985, max_depth=7,
              min_child_samples=1073, min_data_in_leaf=2, n_estimators=480,
              num_leaves=887)

In [358]:
mean_squared_error(lgbm_model.predict(X_val), y_val)

304.4219582188492

In [359]:
submit = pd.DataFrame({'target': lgbm_model.predict(X_val).reshape(-1)})
submit.to_csv('../data/blending/regression/lgbm_preds.csv', index=False)
submit.head()

,target
0,49.622930
1,43.244205
2,45.389293
3,44.769051
4,40.462658


## Обучаем XGBoost

In [361]:
# xgboost не умеет работать с категориальными признаками, так что нужно сделать ohe
X_train = pd.get_dummies(X_train, columns=['car_type', 'fuel_type', 'model'])
X_val = pd.get_dummies(X_val, columns=['car_type', 'fuel_type', 'model'])
X_train.shape

(1752, 36)

In [363]:
params_xgb = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    # 'objective': 'reg:linear',
    # 'eval_metric': 'accuracy'
}

In [298]:
xgb_model = xgb.XGBRegressor(**params_xgb)

In [299]:
xgb_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=100, verbose=25)

[0]	validation_0-rmse:45.80347	validation_1-rmse:45.66103
[25]	validation_0-rmse:20.51602	validation_1-rmse:20.99963
[50]	validation_0-rmse:16.35869	validation_1-rmse:17.68202
[75]	validation_0-rmse:15.41028	validation_1-rmse:17.39982
[99]	validation_0-rmse:14.82798	validation_1-rmse:17.42334


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, early_stopping_rounds=None,
             enable_categorical=False, eta=0.05, eval_metric=None,
             feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, ...)

In [301]:
xgb_model.best_iteration

80

In [302]:
mean_squared_error(xgb_model.predict(X_val), y_val)

302.1528549432632

In [303]:
submit = pd.DataFrame({'target': xgb_model.predict(X_val).reshape(-1)})
submit.to_csv('../data/blending/regression/xgb_preds.csv', index=False)
submit.head()

,target
0,55.741352
1,42.661655
2,42.217468
3,40.969734
4,38.243038


## Блендим модели

### Быстрый и простой способ сбледнить

In [9]:
# 
ensemble = pd.read_csv('../data/blending/regression/catboost_preds.csv')['target'] * 0.5 + \
           pd.read_csv('../data/blending/regression/xgb_preds.csv')['target'] * 0.25 + \
           pd.read_csv('../data/blending/regression/lgbm_preds.csv')['target'] * 0.25
mean_squared_error(ensemble, y_val)

297.04590842596497

### Более элегантный и универсальный способ

In [12]:
import os

In [10]:
weights = {'catboost': 0.5,
           'lgbm': 0.25,
           'xgb': 0.25}

In [18]:
preds = pd.DataFrame()

# соберем единый датафрейм из наших предсказаний
for model_name in ['catboost', 'lgbm', 'xgb']:
    now = pd.read_csv(os.path.join('../data/blending/regression', f'{model_name}_preds.csv')).reset_index()
    now['model'] = model_name
    now['target'] *= weights[model_name]
    preds = pd.concat([preds, now])
    
preds.head()

,index,target,model
0,0,32.963961,catboost
1,1,22.124058,catboost
2,2,21.903983,catboost
3,3,22.830561,catboost
4,4,21.826518,catboost


In [19]:
preds['model'].unique()

array(['catboost', 'lgbm', 'xgb'], dtype=object)

In [20]:
ensemble = preds.groupby('index')['target'].agg('sum')
mean_squared_error(ensemble, y_val)

297.04590842596497

## Выводы

<div class="alert alert-info">

Блендинг это сильный инструмент, который зачастую неплохо поднимаем качество моделей. При этом само сблендивание провести можно вообще в одну строчку, просто взяв среднее моделей.